In [1]:
# Write out CHM json output and forcing config files

In [2]:
import numpy as np
import pandas as pd
import os
import json
import xarray as xr

## OUTPUT

In [3]:
# Load in merged data
# netcdf_file = r'F:\Work\e\Data\Obs\Canada_Project_Sites\CSAS_data\QC\Hourly_QC.nc'
# ds_csas = xr.open_dataset(netcdf_file)
# CRHO_file = r'C:\Users\new356\Google Drive\Nov2014 Data QC Completed Data\Fortress_2014_2016_QC_data\QC_Data_netcdf\CRHO.nc'
# ds_crho_all = xr.open_dataset(CRHO_file)
ds_all = xr.open_dataset(r'F:\Work\e\Data\Obs\Canada_Project_Sites\CSAS_data\QC\Hourly_QC.nc')
# ds_all = ds_all.AirtemperatureA # subset to save memory (just want metadata)
# Courses points
EC_snow_course_in = os.path.normpath(r'F:\Work\e\Data\Obs\Canada_Project_Sites\CSAS_data\EC_Snow_Courses\netcdf\EC_Snow_Courses.nc')
ds_SC = xr.open_dataset(EC_snow_course_in)
# ds_SC = ds_SC.SWE_point
# Output_config_file = os.path.join(r'C:\Users\new356\Model_Output\config','CRHO_output_sta_ALL.json')
Output_config_file = os.path.join(r'C:\Users\new356\Model_Output\config','CSAS_output.json')

In [4]:
ds_all = ds_all[['Lat','Lon','Elevation']]
ds_SC = ds_SC[['Lat','Lon','Elevation']]

In [5]:
ds_all

<xarray.Dataset>
Dimensions:       (staID: 261)
Coordinates:
    Lat           (staID) float64 49.28 49.42 49.8 49.36 50.09 49.67 49.61 ...
    Lon           (staID) float64 -114.4 -114.1 -114.5 -114.5 -114.6 -114.5 ...
    Elevation     (staID) float64 1.52e+03 1.393e+03 1.676e+03 1.97e+03 ...
    station_name  (staID) object 'Westcastle II' ...
    network       (staID) object 'ABE_AGG_HIST' 'ABE_AGG_HIST' ...
  * staID         (staID) object '05AA801' '05AA804' '05AA808' '05AA809' ...
Data variables:
    *empty*

In [6]:
ds = xr.concat([ds_all, ds_SC], dim='staID')

In [7]:
ds

<xarray.Dataset>
Dimensions:       (staID: 283)
Coordinates:
    Lat           (staID) float64 49.28 49.42 49.8 49.36 50.09 49.67 49.61 ...
    Lon           (staID) float64 -114.4 -114.1 -114.5 -114.5 -114.6 -114.5 ...
    Elevation     (staID) float64 1.52e+03 1.393e+03 1.676e+03 1.97e+03 ...
    station_name  (staID) object 'Westcastle II' ...
    network       (staID) object 'ABE_AGG_HIST' 'ABE_AGG_HIST' ...
  * staID         (staID) object '05AA801' '05AA804' '05AA808' '05AA809' ...
Data variables:
    *empty*

In [8]:
# Define the modLandCover for CHM
# Assume all stations are in clearings (1), unless we assing forest(0) or gap(2) here
modLandCover = pd.Series(np.ones(ds.staID.size), index=ds.staID.values)
forest_0_staID = ['LLF', 'UPF']
gap_2_staID    = []
for csta in forest_0_staID:
    modLandCover.loc[csta] = 0
for csta in gap_2_staID:
    modLandCover.loc[csta] = 2   
ds.coords['modLandCover'] = xr.DataArray.from_series(modLandCover).rename({'index':'staID'}).astype(int)

In [9]:
# ds = xr.concat([ds_crho, ds_csas], dim='staID')
# ds

In [10]:
# Have some missing metadata for Bc stations, Need to fix
print("Have some missing metadata for BC stations, Need to fill in. BC hydro still doesn't have the data....lame")
ds = ds.where(ds.Lat.notnull(), drop=True)
ds

Have some missing metadata for BC stations, Need to fill in. BC hydro still doesn't have the data....lame


<xarray.Dataset>
Dimensions:       (staID: 275)
Coordinates:
    Lat           (staID) float64 49.28 49.42 49.8 49.36 50.09 49.67 49.61 ...
    Lon           (staID) float64 -114.4 -114.1 -114.5 -114.5 -114.6 -114.5 ...
    Elevation     (staID) float64 1.52e+03 1.393e+03 1.676e+03 1.97e+03 ...
    station_name  (staID) object 'Westcastle II' ...
    network       (staID) object 'ABE_AGG_HIST' 'ABE_AGG_HIST' ...
  * staID         (staID) object '05AA801' '05AA804' '05AA808' '05AA809' ...
    modLandCover  (staID) int32 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
Data variables:
    *empty*

In [11]:
# Provide lat long boundary to filter stations
lat_r = [50.66, 51.7933333333]
lon_r = [-116.645, -114.769166667]
ds = ds.where( (ds.Lat>=lat_r[0]) & (ds.Lat<=lat_r[1]) & (ds.Lon>=lon_r[0]) & (ds.Lon<=lon_r[1]), drop=True)
ds.staID

<xarray.DataArray 'staID' (staID: 63)>
array(['05BA813', '05BA815', '05BB803', '05BD801', '05BF825', '05BG801',
       '05BG802', '05BG803', '05BH803', '05BJ805', '05BJ806', '05CA805',
       '05CA808', '05CA809', '05CA810', '05CA811', '05CA813', '2A32P', '2C14P',
       'BNS', 'BRP', 'BWH', 'CNT', 'CRG', 'CRN', 'FIRES-B3', 'FIRES-B4',
       'FIRES-B5', 'FIRES-B8', 'FLG', 'FRG', 'FRS', 'FSR', 'HLN', 'HMW', 'LLF',
       'MSC-001', 'MSC-005', 'MSC-028', 'PWL', 'PYT', 'SIB', 'UPC', 'UPF',
       'VVW', '05BA801', '05BA802', '05BA806', '05BA808', '05BA810', '05BA812',
       '05BA813', '05BA814', '05BC801', '05BD801', '05BF821', '05CA806',
       '05CA807', '2A03A', '2A07', '2C05', '2C14', '2C15'], dtype=object)
Coordinates:
    Lat           (staID) float64 51.71 51.46 51.08 51.45 50.79 51.32 51.29 ...
    Lon           (staID) float64 -116.5 -116.1 -115.8 -115.8 -115.1 -115.0 ...
    Elevation     (staID) float64 2.031e+03 2.24e+03 2.23e+03 2.18e+03 ...
    station_name  (staID) object

In [24]:
np.sort(ds.staID.values)

array(['05BA801', '05BA802', '05BA806', '05BA808', '05BA810', '05BA812',
       '05BA813', '05BA813', '05BA814', '05BA815', '05BB803', '05BC801',
       '05BD801', '05BD801', '05BF821', '05BF825', '05BG801', '05BG802',
       '05BG803', '05BH803', '05BJ805', '05BJ806', '05CA805', '05CA806',
       '05CA807', '05CA808', '05CA809', '05CA810', '05CA811', '05CA813',
       '2A03A', '2A07', '2A32P', '2C05', '2C14', '2C14P', '2C15', 'BNS',
       'BRP', 'BWH', 'CNT', 'CRG', 'CRN', 'FIRES-B3', 'FIRES-B4',
       'FIRES-B5', 'FIRES-B8', 'FLG', 'FRG', 'FRS', 'FSR', 'HLN', 'HMW',
       'LLF', 'MSC-001', 'MSC-005', 'MSC-028', 'PWL', 'PYT', 'SIB', 'UPC',
       'UPF', 'VVW'], dtype=object)

In [12]:
# Find stations missing Elevation data (or is zero (wrong))
X = ds.Elevation.where(ds.Elevation.isnull(), drop=True)
list(zip(X.station_name.values,X.staID.values))

[]

In [13]:
# "CanadianRidge":
#     {
#     "file":"CRG_out.txt",
#    "longitude":  -115.206041,
#    "latitude": 50.821573,
#    "type": "timeseries"
#     },

array([-116.47875, -116.47875])

In [30]:
c_ds = ds.sel(staID='05BA813')


2

In [34]:
# Loop through stations and make json format
meta = {}
for csta in list(set(ds.staID.values)):
    c_ds = ds.sel(staID=csta)
    # check for duplicates
    if c_ds.Lat.values.size>1: # We found a duplicate
        c_ds = c_ds.isel(staID=0) # take the first one
        print("found duplicate staID:",csta," Using first one.")
    meta[csta] = {'file':csta+'_out.txt',
    "longitude": c_ds.Lon.values.item(),
    "latitude":  c_ds.Lat.values.item(),
    "elevation": c_ds.Elevation.values.item(),
    "type": "timeseries",
    "modLandCover": c_ds.modLandCover.values.item(),
    }


found duplicate staID: 05BA813  Using first one.
found duplicate staID: 05BD801  Using first one.


In [38]:
print("Found ",len(meta)," stations.")

Found  61  stations.


In [35]:
meta

{'05BA801': {'elevation': 1580.0,
  'file': '05BA801_out.txt',
  'latitude': 51.416667,
  'longitude': -116.183333,
  'modLandCover': 1,
  'type': 'timeseries'},
 '05BA802': {'elevation': 1615.0,
  'file': '05BA802_out.txt',
  'latitude': 51.433333,
  'longitude': -116.166667,
  'modLandCover': 1,
  'type': 'timeseries'},
 '05BA806': {'elevation': 2030.0,
  'file': '05BA806_out.txt',
  'latitude': 51.416667,
  'longitude': -116.233333,
  'modLandCover': 1,
  'type': 'timeseries'},
 '05BA808': {'elevation': 1740.0,
  'file': '05BA808_out.txt',
  'latitude': 51.416667,
  'longitude': -116.216667,
  'modLandCover': 1,
  'type': 'timeseries'},
 '05BA810': {'elevation': 2190.0,
  'file': '05BA810_out.txt',
  'latitude': 51.466667,
  'longitude': -116.1,
  'modLandCover': 1,
  'type': 'timeseries'},
 '05BA812': {'elevation': 2230.0,
  'file': '05BA812_out.txt',
  'latitude': 51.316667,
  'longitude': -116.216667,
  'modLandCover': 1,
  'type': 'timeseries'},
 '05BA813': {'elevation': 2031.0,

In [39]:
# Write out json file with metadata for these points
with open(Output_config_file, 'w') as outfile:
    json.dump(meta, outfile,indent=4, sort_keys=True)

In [40]:
Output_config_file

'C:\\Users\\new356\\Model_Output\\config\\CSAS_output.json'